In [1]:
using ParserCombinator
using Base.Test
using AutoHashEquals
using AbstractTrees
using DataStructures

import ParserCombinator: execute, success, failure

In [2]:
@auto_hash_equals immutable EqualsOneOf{LONGEST}<:Matcher
    name::Symbol
    strings::Trie
    function EqualsOneOf(strings) 
        new(:EqualsOneOf, Trie(strings))
    end
end

EqualsOneOf(strings) = EqualsOneOf{true}(strings)

always_print(::EqualsOneOf) = true

always_print (generic function with 1 method)

In [3]:
function find_matched_positions(kk::Config, strings::Trie, ii::Int)
    Task() do 
        node = strings
        while !done(kk.source, ii)
            char, ii = next(kk.source, ii)
            if haskey(node.children, char)
                node = node.children[char]
                if node.is_key
                    produce(ii-1) #String actually ended the index before
                end
            else
                break
            end
        end
    end
end


order_match_positions(mm::EqualsOneOf{true}, iis) = reverse(collect(iis))
order_match_positions(mm::EqualsOneOf{false}, iis) = (iis)

order_match_positions (generic function with 2 methods)

In [4]:
@auto_hash_equals immutable EqualsOneOfState{I,S}<:State
    start_pos::Int #Start of the matchs
    matches_end_iis::I #Iterator of matched strings 
    matches_state::S  # State for that iterator
end

In [5]:
function execute(kk::Config, mm::EqualsOneOf, ss::Clean, ii::Int)
    start_pos=ii
    found_end_pos = find_matched_positions(kk, mm.strings, ii)
    match_positions = order_match_positions(mm, found_end_pos)
    child_state = EqualsOneOfState(start_pos, match_positions, start(match_positions))
    execute(kk, mm, child_state, start_pos)
end

function execute(kk::Config, mm::EqualsOneOf, ss::EqualsOneOfState, ii)
    if done(ss.matches_end_iis, ss.matches_state)
        FAILURE
    else
        end_pos, next_matches_state = next(ss.matches_end_iis, ss.matches_state)
        next_ss = EqualsOneOfState(ss.start_pos, ss.matches_end_iis, next_matches_state)
        item = SubString(kk.source, ss.start_pos, end_pos)
        results = Any[item]
        source_state = end_pos+1
        Success(next_ss, source_state, results)
    end
end

execute (generic function with 42 methods)

In [6]:
?Base.Generator

```
Generator(f, iter)
```

Given a function `f` and an iterator `iter`, construct an iterator that yields the values of `f` applied to the elements of `iter`. The syntax `f(x) [if cond(x)::Bool] for x in iter` is syntax for constructing an instance of this type. The `[if cond(x)::Bool]` expression is optional and acts as a "guard", effectively filtering out values where the condition is false.


In [7]:
@testset "EqualsOneOf" begin
    @test parse_one("a", EqualsOneOf(["a","b","c"])) == ["a"]
    @test parse_one("abc", EqualsOneOf(["a","b","c"])) == ["a"]
    @test parse_one("x a", E"x " + EqualsOneOf(["a","b","c"])) == ["a"]
    @test_throws ParserException parse_one("z", EqualsOneOf(["a","b","c"]))
    
    
    @test parse_all("abc", EqualsOneOf(["abc","ab","a"])) |> collect == [["abc"], ["ab"], ["a"]]
    @test parse_all("abc", EqualsOneOf{false}(["abc","ab","a"])) |> collect == [["a"], ["ab"], ["abc"]]
    
    @test parse_one("abcd", EqualsOneOf(["abc","ab","a"]) + e"cd") == ["ab", "cd"] #Requires backtracking
    
    
    # Detailed check of endpoint finding behavour
    strings = ["abc", "aabbc", "aabbcc", "aaabc", "aax"];

    @test_throws ParserException parse_one("ab", EqualsOneOf{false}(strings))
    @test parse_one("abc", EqualsOneOf{false}(strings))  == ["abc"]
    @test parse_one("abcx", EqualsOneOf{false}(strings))  == ["abc"]
    @test parse_one("aabbcc", EqualsOneOf{false}(strings))  == ["aabbc"]
    @test parse_one("aabbcx", EqualsOneOf{false}(strings))  == ["aabbc"]


    @test_throws ParserException parse_one("ab", EqualsOneOf(strings))
    @test parse_one("abc", EqualsOneOf(strings))  == ["abc"]
    @test parse_one("abcx", EqualsOneOf(strings))  == ["abc"]
    @test parse_one("aabbcc", EqualsOneOf(strings))  == ["aabbcc"]
    @test parse_one("aabbcx", EqualsOneOf(strings))  == ["aabbc"]
end;

EqualsOneOf: Error During Test
  Test threw an exception of type InterruptException
  Expression: parse_all("abc",EqualsOneOf{false}(["abc","ab","a"])) |> collect == [["a"],["ab"],["abc"]]
  InterruptException:
   in schedule_and_wait(::Task, ::Void) at ./event.jl:110
   in consume(::Task) at ./task.jl:269
   in done at ./task.jl:274 [inlined]
   in grow_to!(::Array{Array{Any,1},1}, ::Task, ::Void) at ./array.jl:363
   in grow_to!(::Array{Union{},1}, ::Task, ::Void) at ./array.jl:372
   in grow_to!(::Array{Any,1}, ::Task) at ./array.jl:357
   in |>(::Task, ::Base.#collect) at ./operators.jl:350
   in macro expansion; at ./In[7]:9 [inlined]
   in macro expansion; at ./test.jl:672 [inlined]
   in anonymous at ./<missing>:?
   in include_string(::String, ::String) at ./loading.jl:441
   in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:169
   in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/eventloop.jl:8
   in (::IJu

LoadError: LoadError: Some tests did not pass: 16 passed, 0 failed, 1 errored, 0 broken.
while loading In[7], in expression starting on line 1

In [8]:
@testset "EqualsOneOf Detailed" begin

end;

Test Summary:        | 
  EqualsOneOf Detailed | No tests


In [9]:
?Success

search: Success success



No documentation found.

**Summary:**

```
immutable ParserCombinator.Success{CS<:ParserCombinator.State,I} <: ParserCombinator.Message
```

**Fields:**

```
child_state :: CS<:ParserCombinator.State
iter        :: I
result      :: Array{Any,1}
```


In [10]:
@which "a"^0

^(s::AbstractString, r::Integer) at strings/types.jl:178

In [11]:
x="abce"
x[1:0] |>typeof


String

In [12]:
?expand

search: expand expanduser macroexpand



```
expand(x)
```

Takes the expression `x` and returns an equivalent expression in lowered form.
